In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_scaled = preprocessing.scale(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), housing_data_scaled]



/home/chris/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
n_epochs = 1000
learning_rate = 0.01
X = tf.placeholder(tf.float32, shape = (None,n+1), name="X")
y = tf.placeholder(tf.float32, shape = (None,1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

batch_size = 100
n_batches = int(np.ceil(m/batch_size))
arr = np.arange(m)
def fetch_batch(epoch,batch_index,batch_size):    
    np.random.seed(epoch)
    np.random.shuffle(arr)
    if ((batch_index * batch_size) + batch_size) <= m:
        random_batch_index = arr[(batch_index * batch_size) : ((batch_index * batch_size) + batch_size)]
    else:
        random_batch_index = arr[(batch_index * batch_size) : m]
    X_batch = scaled_housing_data_plus_bias[[random_batch_index]]
    y_batch = housing.target.reshape(-1, 1)[[random_batch_index]]
    return X_batch, y_batch
#    X_batch = scaled_housing_data_plus_bias

init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            save_path = saver.save(sess,"/tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch,batch_index, batch_size)            
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        
        #sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")
print(best_theta)